### __NLP Disaster Tweets: Sentiment Analysis with Pre-trained Model, Vader__

NLP Reference Links:
> - https://pub.towardsai.net/ai-sustainability-tweets-sentiment-analysis-using-pre-trained-models-8669f8ba70ed
> - NLTK Library of Pre-trained Models: http://www.nltk.org/nltk_data/
> - GitHub for Vedar: https://github.com/cjhutto/vaderSentiment

Other Reference Links:
> - https://www.interviewqs.com/ddi-code-snippets/drop-duplicate-rows-pandas



In [11]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import time

import re
import string
import numpy as np 
import random
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
from plotly import graph_objs as go
import plotly.express as px
import plotly.figure_factory as ff
from collections import Counter

from PIL import Image
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator


import nltk
nltk.download('punkt')
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

from tqdm import tqdm
import os
import nltk
import spacy
import random
from spacy.util import compounding
from spacy.util import minibatch

from collections import defaultdict
from collections import Counter

import keras
from keras.models import Sequential
from keras.initializers import Constant
from keras.layers import (LSTM, 
                          Embedding, 
                          BatchNormalization,
                          Dense, 
                          TimeDistributed, 
                          Dropout, 
                          Bidirectional,
                          Flatten, 
                          GlobalMaxPool1D)
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers.embeddings import Embedding
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from tensorflow.keras.optimizers import Adam

from sklearn.metrics import (
    precision_score, 
    recall_score, 
    f1_score, 
    classification_report,
    accuracy_score
)

# to be able to see multiple ouputs from sungle cell
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

#### __Load Datasets__
  * Only using train data, as it is labeled (i.e. disaster- or non-disaster related tweets), while test data is not labeled

In [2]:
# import train data
train_df = pd.read_csv('/content/train.csv')
train_df.info()
train_df

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7613 entries, 0 to 7612
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   id        7613 non-null   int64 
 1   keyword   7552 non-null   object
 2   location  5080 non-null   object
 3   text      7613 non-null   object
 4   target    7613 non-null   int64 
dtypes: int64(2), object(3)
memory usage: 297.5+ KB


,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1
...,...,...,...,...,...
7608,10869,NaN,NaN,Two giant cranes holding a bridge collapse int...,1
7609,10870,NaN,NaN,@aria_ahrary @TheTawniest The out of control w...,1
7610,10871,NaN,NaN,M1.94 [01:04 UTC]?5km S of Volcano Hawaii. htt...,1
7611,10872,NaN,NaN,Police investigating after an e-bike collided ...,1


#### __Pre-Trained Model: Vader_Lexicon__

In [3]:
#nltk.download()
import nltk
nltk.download('vader_lexicon')

from nltk.sentiment.vader import SentimentIntensityAnalyzer
sentiment = SentimentIntensityAnalyzer()

[nltk_data] Downloading package vader_lexicon to /root/nltk_data...


True

/usr/local/lib/python3.7/dist-packages/nltk/twitter/__init__.py:20: UserWarning: The twython library has not been installed. Some functionality from the twitter package will not be available.
  warnings.warn("The twython library has not been installed. "


In [5]:
# test sentiment analysis
text_test = train_df['text'][0]
print(f'\n{text_test}\n')
sentiment.polarity_scores(text_test)


Our Deeds are the Reason of this #earthquake May ALLAH Forgive us all



{'compound': 0.2732, 'neg': 0.0, 'neu': 0.851, 'pos': 0.149}

In [6]:
# function to convert dictionary output to list output
def dict_converter(dict1):
  dictlist = list()
  for key, value in dict1.items():
    temp = [key,value]
    dictlist.append(temp)
  return dictlist

#example
dict_converter({'compound': 0.2732, 'neg': 0.0, 'neu': 0.851, 'pos': 0.149})

[['compound', 0.2732], ['neg', 0.0], ['neu', 0.851], ['pos', 0.149]]

In [9]:
dict_converter({'compound': 0.2732, 'neg': 0.0, 'neu': 0.851, 'pos': 0.149})[0][1]

0.2732

#### __Sentiment Analysis__

In [17]:
# start time to measure the time of the program execution
start_time = time.time()

tweets = list()

for g in train_df['text']:
  dict_res = dict_converter(sentiment.polarity_scores(g))
  tweets.append([g, dict_res[0][1], dict_res[1][1], dict_res[2][1], dict_res[3][1]])

# make tweets lists above to datarame
tweets = pd.DataFrame(tweets, columns=['text', 'neg', 'neu', 'pos', 'compound'])
tweets.info()
tweets

# print the overall program runtime.
print(f"\n\n--- {(time.time() - start_time)} seconds ---")

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7613 entries, 0 to 7612
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   text      7613 non-null   object 
 1   neg       7613 non-null   float64
 2   neu       7613 non-null   float64
 3   pos       7613 non-null   float64
 4   compound  7613 non-null   float64
dtypes: float64(4), object(1)
memory usage: 297.5+ KB


,text,neg,neu,pos,compound
0,Our Deeds are the Reason of this #earthquake M...,0.000,0.851,0.149,0.2732
1,Forest fire near La Ronge Sask. Canada,0.286,0.714,0.000,-0.3400
2,All residents asked to 'shelter in place' are ...,0.095,0.905,0.000,-0.2960
3,"13,000 people receive #wildfires evacuation or...",0.000,1.000,0.000,0.0000
4,Just got sent this photo from Ruby #Alaska as ...,0.000,1.000,0.000,0.0000
...,...,...,...,...,...
7608,Two giant cranes holding a bridge collapse int...,0.262,0.738,0.000,-0.4939
7609,@aria_ahrary @TheTawniest The out of control w...,0.166,0.834,0.000,-0.5849
7610,M1.94 [01:04 UTC]?5km S of Volcano Hawaii. htt...,0.000,1.000,0.000,0.0000
7611,Police investigating after an e-bike collided ...,0.345,0.655,0.000,-0.7845




--- 2.290241003036499 seconds ---


In [18]:
# start time to measure the time of the program execution
start_time = time.time()

# Add the result of sentiment analysis to train data
train_df['sentiment_neg'] = tweets['neg']
train_df['sentiment_neu'] = tweets['neu']
train_df['sentiment_pos'] = tweets['pos']
train_df['sentiment_compound'] = tweets['compound']

# check
train_df.info()
train_df

# print the overall program runtime.
print(f"\n\n--- {(time.time() - start_time)} seconds ---")

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7613 entries, 0 to 7612
Data columns (total 9 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   id                  7613 non-null   int64  
 1   keyword             7552 non-null   object 
 2   location            5080 non-null   object 
 3   text                7613 non-null   object 
 4   target              7613 non-null   int64  
 5   sentiment_neg       7613 non-null   float64
 6   sentiment_neu       7613 non-null   float64
 7   sentiment_pos       7613 non-null   float64
 8   sentiment_compound  7613 non-null   float64
dtypes: float64(4), int64(2), object(3)
memory usage: 535.4+ KB


,id,keyword,location,text,target,sentiment_neg,sentiment_neu,sentiment_pos,sentiment_compound
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1,0.000,0.851,0.149,0.2732
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1,0.286,0.714,0.000,-0.3400
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1,0.095,0.905,0.000,-0.2960
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1,0.000,1.000,0.000,0.0000
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1,0.000,1.000,0.000,0.0000
...,...,...,...,...,...,...,...,...,...
7608,10869,NaN,NaN,Two giant cranes holding a bridge collapse int...,1,0.262,0.738,0.000,-0.4939
7609,10870,NaN,NaN,@aria_ahrary @TheTawniest The out of control w...,1,0.166,0.834,0.000,-0.5849
7610,10871,NaN,NaN,M1.94 [01:04 UTC]?5km S of Volcano Hawaii. htt...,1,0.000,1.000,0.000,0.0000
7611,10872,NaN,NaN,Police investigating after an e-bike collided ...,1,0.345,0.655,0.000,-0.7845




--- 0.03076481819152832 seconds ---


In [20]:
# drop duplicate tweets and check shape before/after

# start time to measure the time of the program execution
start_time = time.time()

# before dropping duplicates
print('\n', train_df.shape)
print('\n')

# drop a duplicate row, based on column name
train_df = train_df.drop_duplicates(subset='text', keep="first")

# after dropping duplicates
print('\n', train_df.shape)

# check
train_df.info()
train_df

# print the overall program runtime.
print(f"\n\n--- {(time.time() - start_time)} seconds ---")


 (7613, 9)



 (7503, 9)
<class 'pandas.core.frame.DataFrame'>
Int64Index: 7503 entries, 0 to 7612
Data columns (total 9 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   id                  7503 non-null   int64  
 1   keyword             7447 non-null   object 
 2   location            5021 non-null   object 
 3   text                7503 non-null   object 
 4   target              7503 non-null   int64  
 5   sentiment_neg       7503 non-null   float64
 6   sentiment_neu       7503 non-null   float64
 7   sentiment_pos       7503 non-null   float64
 8   sentiment_compound  7503 non-null   float64
dtypes: float64(4), int64(2), object(3)
memory usage: 586.2+ KB


,id,keyword,location,text,target,sentiment_neg,sentiment_neu,sentiment_pos,sentiment_compound
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1,0.000,0.851,0.149,0.2732
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1,0.286,0.714,0.000,-0.3400
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1,0.095,0.905,0.000,-0.2960
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1,0.000,1.000,0.000,0.0000
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1,0.000,1.000,0.000,0.0000
...,...,...,...,...,...,...,...,...,...
7604,10863,NaN,NaN,#WorldNews Fallen powerlines on G:link tram: U...,1,0.249,0.751,0.000,-0.6841
7605,10864,NaN,NaN,on the flip side I'm at Walmart and there is a...,1,0.122,0.878,0.000,-0.4939
7606,10866,NaN,NaN,Suicide bomber kills 15 in Saudi security site...,1,0.328,0.574,0.098,-0.7650
7608,10869,NaN,NaN,Two giant cranes holding a bridge collapse int...,1,0.262,0.738,0.000,-0.4939




--- 0.029552221298217773 seconds ---


#### __Order Tweets by the scores from the most positive to the most negative__

In [22]:
# start time to measure the time of the program execution
start_time = time.time()

train_df = train_df.sort_values(['sentiment_compound'], ascending=False)
train_df

# print the overall program runtime.
print(f"\n\n--- {(time.time() - start_time)} seconds ---")

,id,keyword,location,text,target,sentiment_neg,sentiment_neu,sentiment_pos,sentiment_compound
6992,10028,twister,NaN,Check out 'Want Twister Tickets AND A VIP EXPE...,0,0.000,0.402,0.598,0.9730
6534,9345,survived,"Texas, USA",@thoutaylorbrown I feel like accidents are jus...,0,0.049,0.412,0.539,0.9564
6292,8989,storm,NaN,TodayÛªs storm will pass; let tomorrowÛªs li...,1,0.000,0.509,0.491,0.9471
3163,4541,emergency,"Renfrew, Scotland",@batfanuk we enjoyed the show today. Great fun...,0,0.084,0.354,0.563,0.9423
6778,9710,tragedy,NaN,Maaaaan I love Love Without Tragedy by @rihann...,0,0.000,0.381,0.619,0.9394
...,...,...,...,...,...,...,...,...,...
6407,9159,suicide%20bomber,Worldwide,17 killed in SÛªArabia mosque suicide bombing...,1,0.579,0.421,0.000,-0.9552
6393,9137,suicide%20bomb,Worldwide,? 19th Day Since 17-Jul-2015 -- Nigeria: Suici...,1,0.583,0.417,0.000,-0.9595
6411,9166,suicide%20bomber,NaN,Suicide bomber kills 15 in Saudi security site...,1,0.536,0.400,0.064,-0.9623
6414,9172,suicide%20bomber,NaN,@Abu_Baraa1 Suicide bomber targets Saudi mosqu...,1,0.542,0.458,0.000,-0.9686




--- 0.01908588409423828 seconds ---


In [23]:
# export train data with sentiment analysis above
train_df.to_csv('sentiment_train_data.csv', index=False)